In [12]:
from langchain_huggingface import HuggingFaceEmbeddings
import duckdb
import pandas as pd
import tqdm as notebook_tqdm
import os,sys


# Notebook to build a RAG pipeline for UTMB data
The idea is to build an Agent system that can answer questions about UTMB races. For this we will use an hybrid approach:
1. If the questions is answerable from the db then we will use an LLM to generate a query and query the db. 
    example of question: "What race offers a 50km run in June?" --> Easily queriable. 
2. If it is a more vague question, we will use a RAG approach. 
    example of question: "What is the most challenging race? " --> Not easily queriable.

The agent will then decide which approach to use based on the question. A potential 3rd approach could be to use a combination of both, query the db to filter the resutls and then use the RAG approach to answer the question.
    example of question: "What is the most challenging 50km race that I can do over the summer?" --> This could be answered by first querying the db to get all the 50k runs in summer and then using the RAG approach to answer the question.

In [3]:



class DuckDBRetriever:
    def __init__(self,db_path:str, table_name:str, embedding_model:HuggingFaceEmbeddings | str, vector_column:str='embeddings',description_column:str='description'):
        
        # db connection 
        self.db_conn = duckdb.connect(database=db_path, read_only=True)
        # db descriptions and tables 
        self.table_name = table_name
        self.vector_column = vector_column
        self.description_column = description_column

        #embedding model
        self.embedding_model = self.set_embeder(embedding_model)
    
    def set_embeder(self, embedding_model:HuggingFaceEmbeddings | str)-> HuggingFaceEmbeddings:
        """Set the embedding model to be used for vectorization."""
        if isinstance(embedding_model, str):
            embedding = HuggingFaceEmbeddings(model_name=embedding_model, model_kwargs={"device": "cpu"})
        else:
            embedding = embedding_model
        return embedding
    
    def _embed_query(self, query:str) -> list:
        """Embed the query using the embedding model."""
        query_final = "query: " + query
        return self.embedding_model.embed_query(query_final)
    
    def retrieve(self, query:str, top_k:int=5):
        embed_query = self._embed_query(query)
        self.db_conn.sql(f""" SELECT * ,  """)


In [58]:
model_name = 'intfloat/e5-small-v2'
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={"device": "cpu"})
db = duckdb.connect("../data_test/utmb_db.duckdb")

In [59]:
db.sql("select id from UTMB")

┌────────────┐
│     id     │
│   int64    │
├────────────┤
│          1 │
│          2 │
│          3 │
│          4 │
│          5 │
│          6 │
│          7 │
│          8 │
│          9 │
│         10 │
│          · │
│          · │
│          · │
│         40 │
│         41 │
│         42 │
│         43 │
│         44 │
│         45 │
│         46 │
│         47 │
│         48 │
│         49 │
├────────────┤
│  49 rows   │
│ (20 shown) │
└────────────┘

In [60]:
db.sql("DESCRIBE all tables")

┌──────────┬─────────┬─────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [61]:
data = db.sql("SELECT * FROM UTMB").df()

In [62]:
data.head()

,id,name,date_confirmed,country,city,image,link,distance_5,distance_10,distance_15,...,start_day,end_day,month,year,duration,latitude,longitude,description,embeddings,metadata
0,1,Nice Côte d'Azur,True,France,Nice(06),https://res.cloudinary.com/kavval/image/upload...,https://www.finishers.com/en/event/nice-cote-d...,False,False,False,...,26.0,28.0,9.0,2025.0,2.0,43.700936,7.268391,passage: Nice Côte d'Azur takes place in Nice(...,"[-0.06962153, 0.042183343, 0.055064175, 0.0448...","{ ""name"": ""Nice Côte d'Azur"" }"
1,2,Kaçkar,True,Republic of Türkiye,Ayder,https://res.cloudinary.com/kavval/image/upload...,https://www.finishers.com/en/event/kackar-by-u...,False,False,False,...,27.0,27.0,9.0,2025.0,1.0,40.952520,41.102051,"passage: Kaçkar takes place in Ayder, Republic...","[-0.04515943, 0.010894476, 0.03974204, 0.05890...","{ ""name"": ""Kaçkar"" }"
2,3,KAT100 Austria,True,Austria,Kitzbühel,https://res.cloudinary.com/kavval/image/upload...,https://www.finishers.com/en/event/kat100-utmb,False,True,False,...,7.0,9.0,8.0,2025.0,2.0,47.446359,12.391147,passage: KAT100 Austria takes place in Kitzbüh...,"[-0.052351978, 0.03653495, 0.035622302, 0.0325...","{ ""name"": ""KAT100 Austria"" }"
3,4,UTMB®,True,France,Chamonix(74),https://res.cloudinary.com/kavval/image/upload...,https://www.finishers.com/en/event/utmb,False,False,True,...,25.0,29.0,8.0,2025.0,4.0,33.521298,-117.701083,"passage: UTMB® takes place in Chamonix(74), Fr...","[-0.041295115, 0.037600648, 0.04706601, 0.0074...","{ ""name"": ""UTMB®"" }"
4,5,Julian Alps Trail Run,False,Slovenia,Kranjska Gora,https://res.cloudinary.com/kavval/image/upload...,https://www.finishers.com/en/event/julian-alps...,False,True,True,...,NaN,NaN,9.0,2025.0,NaN,46.485132,13.784396,passage: Julian Alps Trail Run takes place in ...,"[-0.079987764, 0.03809658, -0.018317556, 0.074...","{ ""name"": ""Julian Alps Trail Run"" }"


In [63]:
data.columns

Index(['id', 'name', 'date_confirmed', 'country', 'city', 'image', 'link',
       'distance_5', 'distance_10', 'distance_15', 'distance_20',
       'distance_25', 'distance_30', 'distance_35', 'distance_40',
       'distance_45', 'distance_50', 'distance_55', 'distance_60',
       'distance_65', 'distance_70', 'distance_75', 'distance_80',
       'distance_85', 'distance_90', 'distance_95', 'distance_100',
       'distance_105', 'distance_110', 'distance_115', 'distance_120',
       'distance_125', 'distance_130', 'distance_135', 'distance_140',
       'distance_145', 'distance_150', 'distance_155', 'distance_160',
       'distance_165', 'distance_170', 'distance_175', 'distance_250',
       'distance_300', 'style_Altitude', 'style_Asateam', 'style_Autonomy',
       'style_Capitals/largecities', 'style_Cascade', 'style_Castles',
       'style_Children&JuniorsCourse', 'style_CitytoCity', 'style_Cliffs',
       'style_Cross-border', 'style_Culture&History', 'style_Duo',
       'style_For

## RAG Pipeline 

Our documents are emebedded and stored in a DuckDB database. We will use the `langchain_community.vectorstores.duckdb` to query the embeddings.
The embeddings were generated using `intfloat/e5-small-v2`. So we will keep this. A specificity of that model is that "passage" and "query" have to be added to the text we are embedding. It has been done for the documents, but not for the queries. So we will add it to the queries before embedding them.

In [2]:
model_name = 'intfloat/e5-large-v2'
embeddings_model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={"device": "cpu"})
vector_store = LangChainDuckDB(connection=db, table_name="UTMB", embedding=embeddings_model,vector_key='embeddings')

/Users/thibtd/some fun/utmb_data_eng/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'LangChainDuckDB' is not defined

In [65]:
question = "query: a scenic but tough race in switzerland"

embedded = embeddings_model.embed_query(question)


/Users/thibtd/some fun/utmb_data_eng/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [66]:
a = db.execute("""SELECT *,array_distance(embeddings,cast(? AS FLOAT[384])) AS distance  FROM UTMB ORDER BY distance DESC LIMIT 5""",parameters=[embedded]).fetchdf()

In [67]:
a

,id,name,date_confirmed,country,city,image,link,distance_5,distance_10,distance_15,...,end_day,month,year,duration,latitude,longitude,description,embeddings,metadata,distance
0,26,Trail Running Festival Desert Rats,False,United States,Fruita(CO),https://res.cloudinary.com/kavval/image/upload...,https://www.finishers.com/en/event/trail-runni...,False,True,False,...,NaN,4.0,2026.0,NaN,39.158870,-108.728988,passage: Trail Running Festival Desert Rats ta...,"[-0.017592195, -0.009952028, 0.07496054, 0.043...","{ ""name"": ""Trail Running Festival Desert Rats"" }",0.710480
1,31,Quindío Trail Colombia,False,Colombia,Buena Vista,https://res.cloudinary.com/kavval/image/upload...,https://www.finishers.com/en/event/quindio-tra...,False,False,True,...,NaN,5.0,2026.0,NaN,42.744581,-95.112752,passage: Quindío Trail Colombia takes place in...,"[-0.097181804, 0.011473633, 0.06027981, -0.001...","{ ""name"": ""Quindío Trail Colombia"" }",0.710415
2,49,Quito Trail Ecuador,False,Ecuador,Quito,https://res.cloudinary.com/kavval/image/upload...,https://www.finishers.com/en/event/quito-trail...,True,False,True,...,NaN,8.0,2026.0,NaN,-0.220164,-78.512327,passage: Quito Trail Ecuador takes place in Qu...,"[-0.06586352, 0.018734813, 0.058759533, 0.0160...","{ ""name"": ""Quito Trail Ecuador"" }",0.706034
3,6,Paraty Brazil,False,Brazil,Paraty,https://res.cloudinary.com/kavval/image/upload...,https://www.finishers.com/en/event/paraty-braz...,False,False,False,...,NaN,9.0,2025.0,NaN,-23.141541,-44.761080,"passage: Paraty Brazil takes place in Paraty, ...","[-0.07758545, 0.054810338, 0.058911953, 0.0320...","{ ""name"": ""Paraty Brazil"" }",0.705857
4,20,Hoka Chiang Mai Thaïland,False,Thailand,Chiang Mai,https://res.cloudinary.com/kavval/image/upload...,https://www.finishers.com/en/event/hoka-chiang...,False,True,False,...,NaN,12.0,2025.0,NaN,18.788278,98.985880,passage: Hoka Chiang Mai Thaïland takes place ...,"[-0.048644, 0.035381272, 0.034256756, 0.050821...","{ ""name"": ""Hoka Chiang Mai Thaïland"" }",0.704122


In [76]:
db.sql(f"""select * from UTMB where id == 5 """)


┌───────┬───────────────────────┬────────────────┬──────────┬───────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────┬────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬────────────────┬───────────────┬────────────────┬────────────────────────────┬───────────────┬───────────────┬──────────────────────────────┬───────────

In [55]:
db.close()

## START with a text only RAG approach, we are embedding here so that we can try multiple models.

In [10]:
from sentence_transformers import SentenceTransformer
import torch

In [15]:
torch.mps.empty_cache()

In [9]:
qwen_3_emb = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B",
                                      tokenizer_kwargs={"padding_side": "left"})

In [13]:
text_df = pd.read_csv("../data_test/utmb_text.csv")

In [ ]:
#text_df['description']= text_df['description'].apply(lambda x: "passage: "+x)

In [ ]:
#text_df['embeddings'] = embeddings_model.embed_documents(text_df['description'].tolist())

/Users/thibtd/some fun/utmb_data_eng/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [16]:
text_df['embeddings'] = qwen_3_emb.encode(text_df['description'].tolist())

RuntimeError: MPS backend out of memory (MPS allocated: 7.59 GB, other allocations: 1.45 GB, max allowed: 9.07 GB). Tried to allocate 60.74 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [123]:
text_df.head()

,id,name,distance,description,embeddings
0,1,Nice Côte d'Azur,20,passage: On the trails along the Nice coastlin...,"[0.014466832391917706, -0.016960715875029564, ..."
1,2,Nice Côte d'Azur,55,passage: An Azure epic\nStarting at the mythic...,"[-0.004987615160644054, -0.04516489803791046, ..."
2,3,Nice Côte d'Azur,105,passage: The discovery of Nice County\nFrom th...,"[-0.004149080254137516, -0.016334405168890953,..."
3,4,Nice Côte d'Azur,160,passage: From the Mercantour to the French Riv...,"[-0.015999216586351395, -0.032558683305978775,..."
4,5,Kaçkar,20,passage: KAÇKAR 20K\nYou’ll find yourself runn...,"[0.004555487539619207, -0.056459106504917145, ..."


In [124]:
db.sql("""
                 INSTALL vss;
                LOAD vss;
                 DROP TABLE UTMB_text""")

In [125]:
db.sql("""
             INSTALL vss;
             LOAD vss;
             set hnsw_enable_experimental_persistence = true;
             CREATE TABLE UTMB_text AS SELECT  * EXCLUDE (embeddings),
    CAST(embeddings AS FLOAT[1024]) AS embeddings FROM text_df;""")
db.sql("""CREATE INDEX cos_idx_text ON UTMB_text USING HNSW(embeddings)
            WITH (metric = 'cosine');""")

In [126]:
db.sql("show all tables")

┌──────────┬─────────┬───────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [132]:
query = "query: The race with the most elevation gain in the world"
quemb = embeddings_model.embed_query(query)

In [133]:
a = db.execute("""SELECT *,array_distance(embeddings,cast(? AS FLOAT[1024])) AS distance_sim  FROM UTMB_text ORDER BY distance_sim DESC LIMIT 5""",parameters=[quemb]).fetchdf()

In [134]:
a

,id,name,distance,description,embeddings,distance_sim
0,3,Nice Côte d'Azur,105,passage: The discovery of Nice County\nFrom th...,"[-0.0041490803, -0.016334405, 0.008838155, -0....",0.730978
1,22,Julian Alps Trail Run,25,passage: The 25 km trail is a circular route a...,"[0.042338386, -0.033658, 0.02496297, 0.0088163...",0.705910
2,23,Julian Alps Trail Run,60,passage: The 25 km trail is a circular route a...,"[0.04316184, -0.033021536, 0.024917344, 0.0092...",0.705729
3,8,KAT100 Austria,10,passage: This discipline leads over hill and d...,"[0.010292007, -0.049937394, -0.0009284914, 0.0...",0.698520
4,21,Julian Alps Trail Run,15,passage: The first part of the trail takes you...,"[0.04401171, -0.03848868, 0.012884852, -0.0051...",0.687496
